In [31]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [32]:
from sklearn.pipeline import make_pipeline

In [33]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green-taxi-duration")

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1749787902109, experiment_id='1', last_update_time=1749787902109, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [34]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [35]:
import os

os.getcwd()

'd:\\Data_Engineering\\Docker\\DockerProject\\mlops-zoomcamp-2025\\04-deployment\\web-service-mlflow'

In [36]:
import requests

# Ensure the data directory exists
os.makedirs("data", exist_ok=True)

urls = [
    "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet",
    "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet"
]
for url in urls:
    filename = url.split("/")[-1]
    with open(os.path.join(f"data/{filename}"), "wb") as f:
        f.write(requests.get(url).content)

In [37]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [38]:
from sklearn.metrics import root_mean_squared_error

with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = root_mean_squared_error(y_pred, y_val)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


2025/06/12 22:16:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run honorable-wasp-527 at: http://127.0.0.1:5000/#/experiments/1/runs/21f992fa55c044a6a8e057b7da7a8765
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [39]:
from mlflow.tracking import MlflowClient


In [40]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

experiment = client.get_experiment_by_name("green-taxi-duration")
runs = client.search_runs(experiment_ids=[experiment.experiment_id], order_by=["start_time DESC"], max_results=1)
RUN_ID = runs[0].info.run_id